#  **SM64 Speedruns - A Python Test**

\** **Ignore any redundant / unnecessary comments, they're mainly just notes for me because I am a noob :]**

Learning Python for data analysis / science and wanted to test what I've learned so far on a personal project using a dataset from Kaggle. (https://www.kaggle.com/code/mcpenguin/super-mario-64-speedruns-data-collection)
\
\
I also haven't used git in a while so this is also sort of a guinea pig for re-learning that.

In [1]:
# Connecting to Google Drive to access dataset files saved there.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **SQLite Connection**

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import csv, sqlite3, os
import openpyxl

# Connection Object to establish connection to a sqlite3 database.
connObj = sqlite3.connect('SPEEDRUNS.db')

# Loading the SQL Magic extension to be able to run SQL queries through / within Python code (available via Pandas)
%load_ext sql
%sql sqlite:///SPEEDRUNS.db

# Cursor Object against connObj to access controller functions like fetchall(), open(), execute(), close(), etc.
# Don't really need this I think since I'm using SQL Magic, but it may be useful for later.
cursorObj = connObj.cursor()

 * sqlite:///SPEEDRUNS.db
(sqlite3.OperationalError) no such table: ALL_CAT_SPEEDRUNS
[SQL: SELECT COUNT(*) FROM ALL_CAT_SPEEDRUNS]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### **Merging Separate .CSV Files Into A Single Pandas DataFrame.**

**Gathering Datasets (.CSV)**

Included the separate read_csv() calls for each dataset .csv file. I was originally going to do this, and then thought of merging them in VBA for fun, but then wanted to do it in Python.

In [3]:
# Assigning the datasets location for SM64 Speedruns to a variable called "repo"
repo = r'/content/drive/MyDrive/Kaggle/Datasets/SM64 Speedruns/'

# Lists files within the specified directory, in this case "repo".
files_in_repo = os.listdir(repo)

# Found online. Didn't really think I would need this, but remembered that I do have a .xlsm file in this directory so this is for fun.
csv_files = [f for f in files_in_repo if f.endswith('.csv')] # This checks to see if each file within the repo's directory ends in '.csv'.

zeroStar = pd.read_csv('/content/drive/MyDrive/Kaggle/Datasets/SM64 Speedruns/data_0 Star.csv')
oneStar = pd.read_csv('/content/drive/MyDrive/Kaggle/Datasets/SM64 Speedruns/data_1 Star.csv')
sixteenStar = pd.read_csv('/content/drive/MyDrive/Kaggle/Datasets/SM64 Speedruns/data_16 Star.csv')
seventyStar = pd.read_csv('/content/drive/MyDrive/Kaggle/Datasets/SM64 Speedruns/data_70 Star.csv')
oneTwentyStar = pd.read_csv('/content/drive/MyDrive/Kaggle/Datasets/SM64 Speedruns/data_120 Star.csv')

# List to hold the list of dataframes / csv files.
df_list = []



---



**Appending Separate .CSV DataFrames to the "df_list" List via For Loop**

Found this online because I wasn't sure of the syntax on doing the loop, and the error handling is good, too.

It all makes sense though - here's my walkthrough:

1.   Looping through the list of `csv_files` within `repo`, assigning each to "`csv`".

2.   The path to the file is created by joining the `repo` path and the .csv filename.

1.   Creating a DataFrame (for each iteration of `csv_files`) using the `read_csv()` function and the `file_path` variable.
2.   The DataFrame is appended to the DataFrame List `df_list`.

1.   `try` / `except` = error handling on the encoding types for the files.

In [4]:
for csv in csv_files:
    file_path = os.path.join(repo, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")



---



**Concatenating the DataFrames and Saving to a Single .CSV File**

In [5]:
# Concat all data into a single DataFrame

complete_df = pd.concat(df_list, ignore_index=True)



---



### **Cleansing / Restructuring Data With *NumPy* And *Pandas***

In [6]:
full_path_to_dataset = os.path.join(repo, 'ALL_CATEGORIES.csv')

if count == 0:
    # Save the final result to a new .csv file (appears in the G Drive folder after 15-30 sec).
    complete_df.to_csv(full_path_to_dataset, index=False)
    allCategoriesDF = pd.read_csv(full_path_to_dataset)
    # Loading the dataframe into the SPEEDRUNS database using the connObj. Specified a table name of "ALL_CAT_SPEEDRUNS".
    allCategoriesDF.to_sql('ALL_CAT_SPEEDRUNS', connObj, if_exists='replace', index=False)
else:
    %sql DELETE FROM ALL_CAT_SPEEDRUNS
    allCategoriesDF = pd.read_csv(full_path_to_dataset)
    # Loading the dataframe into the SPEEDRUNS database using the connObj. Specified a table name of "ALL_CAT_SPEEDRUNS".
    allCategoriesDF.to_sql('ALL_CAT_SPEEDRUNS', connObj, if_exists='replace', index=False)

In [7]:
allCategoriesDF = pd.read_csv(full_path_to_dataset)
allCategoriesDF

,Unnamed: 0,id,place,speedrun_link,submitted_date,primary_time_seconds,real_time_seconds,player_id,player_name,player_country,platform,verified
0,0,yol5kxjy,1,https://www.speedrun.com/sm64/run/yol5kxjy,2023-03-23T06:20:48Z,875.50,875.50,jn32931x,Suigi,Canada,N64,Yes
1,1,zppddvxz,2,https://www.speedrun.com/sm64/run/zppddvxz,2023-11-25T16:02:09Z,881.21,881.21,1xypyovx,Slipperynip,United States,N64,Yes
2,2,y60jnn0y,3,https://www.speedrun.com/sm64/run/y60jnn0y,2022-07-06T19:10:40Z,892.00,892.00,x353dr7j,Finnii602,Germany,VC,Yes
3,3,yjxj5w3z,4,https://www.speedrun.com/sm64/run/yjxj5w3z,2021-12-01T08:19:33Z,893.45,893.45,kjprmwk8,Weegee,United States,N64,Yes
4,4,ywj1gg9z,5,https://www.speedrun.com/sm64/run/ywj1gg9z,2021-12-09T16:44:16Z,895.81,895.81,kj92gmv8,Dowsky,United States,N64,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
5027,496,yd0n6eqz,497,https://www.speedrun.com/sm64/run/yd0n6eqz,2020-12-05T15:12:03Z,7429.00,7429.00,pj0y2m3x,Linkx2,Germany,N64,Yes
5028,497,dy45rk3m,498,https://www.speedrun.com/sm64/run/dy45rk3m,2014-12-17T03:22:18Z,7431.00,7431.00,qxk4pmj0,TPositive,United States,VC,Yes
5029,498,z1pl6e7z,499,https://www.speedrun.com/sm64/run/z1pl6e7z,2022-06-16T09:32:31Z,7444.00,7444.00,8g5kr62j,NaturallyAllen,United States,EMU,Yes
5030,499,y2qngo6y,500,https://www.speedrun.com/sm64/run/y2qngo6y,2019-02-11T01:26:52Z,7445.00,7445.00,qxkm7g7j,meowmix_fan,United States,VC,Yes


In [9]:
# Dropping some unwanted columns.
#
# Note: I'll be remaking the 'id' column since multiple datasets were merged.
cols_to_drop = ['Unnamed: 0',
                  'id',
                  'player_id']

allCategoriesDF.drop(cols_to_drop, inplace=True, axis=1)

sorted(allCategoriesDF.columns)


KeyError: ignored